In [4]:
# filter warnings to not confuse readers
import warnings
warnings.filterwarnings("ignore")

# Python for Excel Users - Part 2
## Modifying and manipulating Excel files

#### Is it for me?
This tutorial series is for everyone who wants to get started working in python instead of Excel, e.g. for automating tasks or improving speed and scalability. Or for just the curious ones.

You're at the right spot when you use Excel to combine various sheets or tables (using formulas like Index, Match, VLookup), use simple mathematical operations like sum and mean, or even use conditional aggregation functions (e.g. sum of transactions per category) or pivot tables.

#### Goal:
In this part we are faced with the task to update a certain Excel file, a sales report, with new data.

#### Requirements:
While some basic experience with Python helps, you don't have to be a a programmer or data scientist to follow this tutorial. Ideally, you should have heard of pandas and jupyter notebook (which is where you're reading this tutorial right now) and spent some time with python yourself. 

But don't get discouraged. Just try to follow along and read up on things we did not cover. You can always come back here. And if you need some personal guidance, we at HOSD Mentoring offer free one-to-one data mentoring at www.hosd-mentoring.com


# 1. The Situation

In part one we assumed you just started as an data analyst in  the sales department of a huge online retailer. In the past few weeks you dug into all kinds of data the business produces. Of course you're also responsible to produce regular **sales reports**.

The company provided a general structure for these reports in form of an Excel template. One of the most important report is the **monthly sales report** since it is the one-stop report for many business leaders for showing the health and development of the business.

This is how the report looks like.

![Excel sales report](img/img1.png)

While this tutorial will be less about doing analytics and more about the process of updating and maintaining a given Excel report, a few strategic questions come up that could lead to useful extensions of the current report.

#### A few strategic questions:
- How did certain brands develop (absolute and relative) over time?
- Development of product categories (absolute and relative)
- Top 5 Growth and Loss products

Since it's August, the company asks us to update the report with the July data shown below.

![Sales data for July 2021](img/img2.png)

We could do this completely in Excel, for example by creating pivot tables, one for brands and one for category, and then copy over the aggregated sales numbers to the summary sheet for July. 

However, while this manual work with only four brands and categories and less than 20 products is approachable, the company is expected to grow its offering significantly within the next months. So it's reasonable to start setting up a more scalable and automated process.

## 2. Setting Everything Up

We already know how to use pandas to read csv files using the method pd.read_csv. Luckily, there's a method for reading Excel files in pandas too: [pd.read_excel()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html). 

Let's see how it looks.

In [2]:
import pandas as pd

# Read sales for july
sales_july = pd.read_csv("data/sales_july_export.csv")
sales_july.head()

,Month,Product Name,Brand,Product Category,Sales
0,July,Blue Ball,Brand A,Category I,68
1,July,Green Ball,Brand A,Category I,123
2,July,Red Ball,Brand B,Category I,579
3,July,Orange Ball,Brand C,Category I,1293
4,July,Pink Ball,Brand C,Category I,1098


In [29]:
# Read excel file with pandas
monthly_sales_report = pd.read_excel("data/Monthly Sales Report.xlsx")
monthly_sales_report.head(15)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,MONTHLY SALES REPORT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Date,July 2021,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contact:,contact@company.com,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,OVERALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,January,February,March,April,May,June,July,August,September,October,November,December,Full Year
6,NaN,Brand A,202,290,189,478,678,679,0,0,0,0,0,0,2516
7,NaN,Brand B,5609,5790,6519,7901,6972,5793,0,0,0,0,0,0,38584
8,NaN,Brand C,1256,4627,2415,671,4678,3561,0,0,0,0,0,0,17208
9,NaN,Brand D,527,521,467,421,652,256,0,0,0,0,0,0,2844


Urgh! This looks weird and not at all like the report when we open it in Excel.

The reason is that pandas simply tries to show the content of each cell as a value in a dataframe. Thus, formatting gets ignored as well as merged and combined cells.

Normally, to work with Excel data in pandas requires to clean up the dataframe, e.g. by removing all the unnecessary rows and columns (like headlines). This process is called slicing and you can read more about it in pandas' official documentation for [Indexing and Selecting data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html). 

For example, this is how the brand table would look like.

In [35]:
# create copy of full report df to slice
monthly_brand_df = monthly_sales_report.copy()

# use first row as header (alternative: set header parameter when reading excel file)
monthly_brand_df.columns = monthly_brand_df.iloc[5]

# only keep relevant rows with data for brand sales
monthly_brand_df = monthly_brand_df[6:11]

# first column is always empty as it exists purely for formatting reasons; drop it
monthly_brand_df= monthly_brand_df.iloc[:,1:]

# reset index after slicing
monthly_brand_df.reset_index(drop=True, inplace=True)

# change column names
monthly_brand_df.columns = ['Brand', 'January', 'February', 'March', 'April',
                            'May', 'June', 'July', 'August', 'September',
                            'October', 'November',  'December', 'Full Year']

monthly_brand_df

,Brand,January,February,March,April,May,June,July,August,September,October,November,December,Full Year
0,Brand A,202,290,189,478,678,679,0,0,0,0,0,0,2516
1,Brand B,5609,5790,6519,7901,6972,5793,0,0,0,0,0,0,38584
2,Brand C,1256,4627,2415,671,4678,3561,0,0,0,0,0,0,17208
3,Brand D,527,521,467,421,652,256,0,0,0,0,0,0,2844
4,Total,7594,11228,9590,9471,12980,10289,0,0,0,0,0,0,61152


However, in this scenario we are not interesting in working with the data but in **updating the report with new data**. In fact, there are only a few cells that we need to touch, namely the cells corresponding to the month of July.

Thus, assuming that the structure of the report stays constant, we are looking for a way to fill specified cells with specified values.

**Our approach is therefore:**
- Step 1: Calculate the July sales on brand and product category level (two summary tables)
- Step 2: Write the data to the Excel report and save the new updated version

### 2.1. Calculate July summary tables (brand and product category level)

### 2.2. Write data to Excel report

# 4. Summary

Now that we learned that, thanks to pandas, working in python is a real alternative to Excel, we got a bit carried away and calculated all sorts of KPIs based on aggregated data from the three original table. A quick look at the watch tells us what our stomach already knew: it's lunch time!

But before we head down to the company cantine aka. wending machine, we want to save what we did and send it to our colleagues. How do we do that?

We can simply **export dataframes as csv files** and send them to others who prefer to work in Excel.

In [ ]:
# Export dataframe as csv for Excel
products_per_category_df.to_csv("output/products_per_category_df.csv", index=False)

Today we learned a few basic but powerful functions that allow us to work in Python instead of Excel. Here's a summary.

- *pd.read_csv()* can be used to import .csv files as dataframes

- *table_df.head()* shows us a preview of that table as output

- to access a column of table_df, we type *table_df["column_name"]*

- *len(pd.unique(table_df["column_name"]))* gives us the number of unique values in that column

- we can multiply two columns and save the result in a new column using *table_df["new"] = table_df["col1"] * table_df["col2"]*

- Instead of Excel's pivot table, we can aggregate values per category, e.g. count the number of transactions per year or sum up the revenue per order using *table_df.groupby(by=["category"], as_index=False).agg({"col3":"sum"})*

- Instead of using VLOOKUP() in Excel we can merge two tables using *pd.merge(left=table_1_df, right=table_2_df, how="left", on="key_column")*

I suggest you to keep going, play around with this dataset or your own a bit further. Whenever you don't know how to do something best is to check the [pandas documentation](https://pandas.pydata.org/docs/) or to google your specific question ("python datetime from string" or "python pivot table") and see how others do it. Just don't get discouraged and keep on learning! 

The beginning is always tough but stick with it and you will be way more effective and faster than any Excel pro.

!["hosd data mentoring"](img/img6.png)


And in case you need some general guidance on whether Data Analytics and Data Science could be a career path for you, we at [HOSD Mentoring offer free one-to-one mentoring](http://www.hosd-mentoring.com). You can book a session with one of our mentors with just a click on our website. Don't be shy :)